In [81]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale

### SVM (support vector machine) is good for cases where p >> n
i.e. where the number of columns/features/variables is much greater than the number of observations

In our case, we have 1000-1500 lipid features and only 60 total observations (10 mice, 6 samples each)

However, SVM is O(n^2) so classical SVM tends to overwhelm memory after 10,000 rows.

### Questions to answer with machine learning:

1. Is RBG or FBG a better model for blood glucose testing in diabetic nile rats>  
2. Which features are the best predictor of diabetes in nile rats? 
    1. Loadings plot in PCA
    2. Dimensionality reduction 

### Models to consider

1. Time series ... something...
3. Dimensionality reduction
    1. LASSO
    2. Ridge/Elastic net regression
4. Accounting for non-independence in observations
    1. Random Effects modelling within Linear Mixed Models (LMM)

5. From the Mike Snyder paper on multiomics correlations to prediabetes https://www.nature.com/articles/s41586-019-1236-x:
    1. " l1 regularization is used to encourage the sparsity of the learned coefficient. "
    2. Regularization is a penalty that increases as model complexity increases
    3. L1 = Lasso Regression
    4. L2 = Ridge Regression 
        1. Main difference is the penalty term used 
    

In [154]:
fr = pd.read_csv(r'..\data\processed\filtered_raw_data.csv', index_col=0)
file_grouping = pd.read_csv(r'..\data\metadata\file_groupings.csv', index_col=0)
fr

,Unique Identifier,Retention Time (min),Quant Ion,Polarity,Area (max),Identification,Lipid Class,Features Found,20210729_AJ_Toh_RatBloodGlucose_T1060M_20210322_8wk_FBG,20210729_AJ_Toh_RatBloodGlucose_T1060M_20210325_8wk_RBG,...,20210729_AJ_Toh_RatBloodGlucose_T1101M_20210402_9wk_FBG,20210729_AJ_Toh_RatBloodGlucose_T1101M_20210406_9wk_RBG,20210729_AJ_Toh_RatBloodGlucose_T1101M_20210410_10wk_FBG,20210729_AJ_Toh_RatBloodGlucose_T1101M_20210413_10wk_RBG,20210729_AJ_Toh_RatBloodGlucose_T1102M_20210327_8wk_FBG,20210729_AJ_Toh_RatBloodGlucose_T1102M_20210330_8wk_RBG,20210729_AJ_Toh_RatBloodGlucose_T1102M_20210402_9wk_FBG,20210729_AJ_Toh_RatBloodGlucose_T1102M_20210406_9wk_RBG,20210729_AJ_Toh_RatBloodGlucose_T1102M_20210410_10wk_FBG,20210729_AJ_Toh_RatBloodGlucose_T1102M_20210413_10wk_RBG
0,unknown_679.05463_0.536,0.536,679.05463,+,2.408421e+07,unknown,unknown,49,1.145548e+07,1.728814e+07,...,1.231938e+07,1.258442e+07,6.377443e+06,1.771940e+07,1.647246e+07,1.366983e+07,1.568381e+07,1.290439e+07,1.353820e+07,1.360893e+07
1,unknown_641.1087_0.537,0.537,641.10870,+,5.522238e+07,unknown,unknown,36,4.677270e+07,5.522238e+07,...,3.428703e+07,3.415574e+07,2.455848e+07,3.852845e+07,4.172903e+07,6.220326e+06,4.279894e+07,3.789880e+07,3.219858e+07,2.889174e+07
2,unknown_951.15973_0.541,0.541,951.15973,-,3.238113e+07,unknown,unknown,10,1.625939e+07,3.238113e+07,...,1.802672e+07,1.658884e+07,1.349634e+07,1.569916e+07,1.912593e+07,1.216820e+07,2.036861e+07,1.952822e+07,1.953922e+07,1.543462e+07
3,unknown_329.03067_0.542,0.542,329.03067,-,2.882770e+08,unknown,unknown,55,1.290199e+08,6.760284e+07,...,1.575765e+08,2.173414e+08,1.282113e+08,2.882770e+08,1.142664e+08,1.452781e+08,1.763549e+08,2.236196e+08,1.790767e+08,1.651242e+08
4,unknown_969.13147_0.544,0.544,969.13147,-,1.697296e+07,unknown,unknown,14,1.089197e+07,1.697296e+07,...,7.257362e+06,9.879815e+06,6.178661e+06,6.578465e+06,8.023917e+06,4.351532e+06,1.132409e+07,1.040661e+07,8.373267e+06,9.059848e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,unknown_974.9118_20.163,20.163,974.91180,+,4.515337e+06,unknown,unknown,4,6.585959e+05,3.591638e+05,...,3.448085e+05,5.558781e+05,1.153422e+05,3.896841e+05,3.842817e+05,5.605229e+05,3.046913e+05,1.110650e+06,5.464527e+05,7.021282e+05
1482,unknown_593.58716_20.175,20.175,593.58716,+,3.296976e+06,unknown,unknown,19,1.801313e+06,1.867413e+05,...,7.956023e+05,3.804952e+05,4.007204e+05,2.390561e+05,1.291650e+06,2.886770e+05,8.545798e+05,6.331233e+05,1.647502e+06,2.088592e+05
1483,unknown_871.80945_20.196,20.196,871.80945,+,2.595829e+06,unknown,unknown,6,1.053780e+06,7.649254e+04,...,4.949319e+05,1.446089e+05,2.794735e+05,7.948007e+04,8.523295e+05,8.869149e+04,4.884374e+05,1.233201e+05,1.052764e+06,4.261791e+04
1484,unknown_1003.93829_20.435,20.435,1003.93829,+,3.203813e+06,unknown,unknown,4,5.844701e+05,2.637219e+05,...,2.294775e+05,6.374218e+05,4.261154e+04,3.421911e+05,3.890957e+05,7.912981e+05,1.910285e+05,1.147269e+06,4.434897e+05,5.509051e+05


In [40]:
quant_filenames = file_grouping[file_grouping['quant_file'] == True]['File Name'].to_list()
quant_filenames

['20210729_AJ_Toh_RatBloodGlucose_T1076M_20210405_10wk_FBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1062M_20210322_8wk_FBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1093M_20210402_9wk_FBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1060M_20210325_8wk_RBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1091M_20210410_10wk_FBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1092M_20210406_9wk_RBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1093M_20210413_10wk_RBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1082M_20210405_10wk_FBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1060M_20210401_9wk_RBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1101M_20210402_9wk_FBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1062M_20210408_10wk_RBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1091M_20210330_8wk_RBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1074M_20210405_10wk_FBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1101M_20210413_10wk_RBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1076M_20210329_9wk_FBG',
 '20210729_AJ_Toh_RatBloodGlucose_T1092M_20210402_9wk_FBG',
 '20210729_AJ_Toh_RatBloodGlucose

In [74]:
fr[quant_filenames].iloc[0:5, 0:3].T

,0,1,2,3,4
20210729_AJ_Toh_RatBloodGlucose_T1076M_20210405_10wk_FBG,1.388749e+06,3.035806e+06,1.663567e+06,2.215194e+07,6.816564e+05
20210729_AJ_Toh_RatBloodGlucose_T1062M_20210322_8wk_FBG,1.049443e+07,2.537288e+07,1.402438e+07,1.187355e+08,6.906003e+06
20210729_AJ_Toh_RatBloodGlucose_T1093M_20210402_9wk_FBG,9.897531e+06,2.207764e+07,8.735853e+06,1.114405e+08,3.505334e+06


array([[0.        , 0.        , 0.        , ..., 0.1264256 , 0.02761148,
        0.        ],
       [0.4012118 , 0.42802335, 0.40240212, ..., 0.14562449, 0.02091204,
        0.03890562],
       [0.37491124, 0.36488002, 0.23023592, ..., 0.79856239, 0.20566073,
        0.25602656],
       ...,
       [0.73288182, 0.65976443, 0.70277078, ..., 0.02169909, 0.01560124,
        0.03127962],
       [0.95062647, 0.81202148, 0.8139675 , ..., 0.0018939 , 0.09156507,
        0.12729206],
       [0.79733941, 0.67962568, 0.51127367, ..., 0.        , 0.06926706,
        0.07341166]])

In [143]:
fr[quant_filenames].T

,0,1,2,3,4,5,6,7,8,9,...,1471,1472,1473,1479,1480,1481,1482,1483,1484,1486
20210729_AJ_Toh_RatBloodGlucose_T1076M_20210405_10wk_FBG,1.388749e+06,3.035806e+06,1.663567e+06,2.215194e+07,6.816564e+05,8.682616e+06,3.747518e+08,1.902448e+05,6.530761e+05,2.106110e+07,...,3.407580e+05,3.310845e+05,3.574035e+05,8.765036e+04,4.881645e+05,1.149073e+05,2.165170e+05,3.460561e+05,1.032091e+05,1.395636e+04
20210729_AJ_Toh_RatBloodGlucose_T1062M_20210322_8wk_FBG,1.049443e+07,2.537288e+07,1.402438e+07,1.187355e+08,6.906003e+06,8.068469e+07,1.338627e+09,2.933464e+05,4.245591e+06,7.975980e+07,...,1.478305e+05,2.429766e+05,7.498958e+05,1.952125e+05,6.713921e+05,1.696026e+05,6.228681e+05,3.955002e+05,8.184697e+04,8.400065e+04
20210729_AJ_Toh_RatBloodGlucose_T1093M_20210402_9wk_FBG,9.897531e+06,2.207764e+07,8.735853e+06,1.114405e+08,3.505334e+06,7.230815e+07,8.257390e+08,9.069245e+05,4.715529e+06,5.267292e+07,...,2.399226e+06,1.834312e+06,2.817875e+06,2.294994e+06,3.066207e+06,1.137056e+06,2.862648e+06,2.077054e+06,6.709454e+05,4.748974e+05
20210729_AJ_Toh_RatBloodGlucose_T1060M_20210325_8wk_RBG,1.728814e+07,5.522238e+07,3.238113e+07,6.760284e+07,1.697296e+07,1.685820e+08,4.234264e+09,4.231877e+05,5.076570e+06,1.380264e+08,...,1.324342e+06,7.152784e+05,1.606961e+06,3.590850e+05,2.521954e+05,3.591638e+05,1.867413e+05,7.649254e+04,2.637219e+05,1.935145e+05
20210729_AJ_Toh_RatBloodGlucose_T1091M_20210410_10wk_FBG,2.129622e+07,4.634647e+07,1.778254e+07,1.723259e+08,5.381199e+06,1.305017e+08,1.526645e+09,1.184495e+06,5.991201e+06,8.910744e+07,...,2.190917e+05,3.657747e+05,1.001713e+06,3.223449e+05,1.178122e+06,2.653676e+05,1.071320e+06,9.205989e+05,1.714771e+05,1.547650e+05
20210729_AJ_Toh_RatBloodGlucose_T1092M_20210406_9wk_RBG,1.623684e+07,3.333306e+07,1.947128e+07,1.765802e+08,9.870417e+06,1.120618e+08,1.813532e+09,2.953879e+05,6.113765e+06,8.745404e+07,...,3.494946e+05,4.528169e+05,1.057110e+06,3.604638e+05,2.383367e+05,2.885532e+05,1.754275e+05,4.931699e+04,2.014974e+05,1.545304e+05
20210729_AJ_Toh_RatBloodGlucose_T1093M_20210413_10wk_RBG,1.614774e+07,3.011081e+07,1.352241e+07,1.880087e+08,7.580783e+06,8.283663e+07,1.023909e+09,6.737710e+05,5.514658e+06,6.109300e+07,...,5.383021e+05,6.354900e+05,1.156747e+06,3.389015e+05,4.842209e+05,2.995037e+05,3.294683e+05,1.025987e+05,2.503579e+05,1.893252e+05
20210729_AJ_Toh_RatBloodGlucose_T1082M_20210405_10wk_FBG,1.152504e+07,2.861964e+07,1.872223e+07,1.666531e+08,8.089105e+06,8.771581e+07,1.231058e+09,2.710627e+05,6.723898e+06,7.190480e+07,...,1.004726e+05,4.831859e+04,4.891069e+05,4.355368e+04,6.514016e+05,2.383030e+04,7.764236e+05,7.099401e+05,1.516585e+04,2.264335e+04
20210729_AJ_Toh_RatBloodGlucose_T1060M_20210401_9wk_RBG,2.381849e+07,3.699951e+07,9.969525e+06,2.170599e+08,5.828327e+06,7.780203e+07,9.474041e+08,3.177282e+05,7.656925e+06,5.256910e+07,...,8.687319e+05,8.145910e+05,1.648307e+06,5.292559e+05,3.282916e+05,4.957885e+05,2.249304e+05,9.603305e+04,3.785893e+05,2.218873e+05
20210729_AJ_Toh_RatBloodGlucose_T1101M_20210402_9wk_FBG,1.231938e+07,3.428703e+07,1.802672e+07,1.575765e+08,7.257362e+06,1.155075e+08,1.447831e+09,5.197247e+06,6.416476e+06,8.422224e+07,...,7.386505e+05,5.912112e+05,1.242546e+06,6.733549e+05,8.886334e+05,3.448085e+05,7.956023e+05,4.949319e+05,2.294775e+05,1.593148e+04


In [144]:
scaler = MinMaxScaler()# defaults to range of (0, 1)
# Can also use minmax_scale as a one-liner:
# minmax_scale(fr[quant_filenames].T)

fr_scaled = pd.DataFrame(scaler.fit_transform(fr[quant_filenames].T), 
                         columns=fr[quant_filenames].T.columns,
                         index=fr[quant_filenames].T.index)

# print(fr_scaled.shape)
# SVM needs to be in the shape of (observations, features)
# print(fr_scaled.T.shape)
fr_scaled.iloc[:, 0:1].min()
# MinMaxScaler(fr[quant_filenames].iloc[:, 1].values.reshape(-1,1),)

0    0.0
dtype: float64

In [148]:
file_grouping

,File Name,analytical_run_order,fr_name,rat_label,bg_type,quant_file,week,lipidex_file_number
0,20210729_AJ_Toh_RatBloodGlucose_Water_R1,1,20210729_AJ_Toh_RatBloodGlucose_Water_R1.raw (...,NaN,NaN,False,NaN,65
1,20210729_AJ_Toh_RatBloodGlucose_Water_R2,2,20210729_AJ_Toh_RatBloodGlucose_Water_R2.raw (...,NaN,NaN,False,NaN,66
2,20210729_AJ_Toh_RatBloodGlucose_Water_R3,3,20210729_AJ_Toh_RatBloodGlucose_Water_R3.raw (...,NaN,NaN,False,NaN,67
3,20210729_AJ_Toh_RatBloodGlucose_SolventBlank,4,20210729_AJ_Toh_RatBloodGlucose_SolventBlank.r...,NaN,NaN,False,NaN,2
4,20210729_AJ_Toh_RatBloodGlucose_ExtractionBlank,5,20210729_AJ_Toh_RatBloodGlucose_ExtractionBlan...,NaN,NaN,False,NaN,1
...,...,...,...,...,...,...,...,...
62,20210729_AJ_Toh_RatBloodGlucose_T1062M_2021040...,63,20210729_AJ_Toh_RatBloodGlucose_T1062M_2021040...,1062.0,RBG,True,9.0,14
63,20210729_AJ_Toh_RatBloodGlucose_T1101M_2021033...,64,20210729_AJ_Toh_RatBloodGlucose_T1101M_2021033...,1101.0,RBG,True,8.0,54
64,20210729_AJ_Toh_RatBloodGlucose_T1092M_2021033...,65,20210729_AJ_Toh_RatBloodGlucose_T1092M_2021033...,1092.0,RBG,True,8.0,42
65,20210729_AJ_Toh_RatBloodGlucose_T1092M_2021041...,66,20210729_AJ_Toh_RatBloodGlucose_T1092M_2021041...,1092.0,RBG,True,10.0,46


In [149]:
fr_scaled.index

Index(['20210729_AJ_Toh_RatBloodGlucose_T1076M_20210405_10wk_FBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1062M_20210322_8wk_FBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1093M_20210402_9wk_FBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1060M_20210325_8wk_RBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1091M_20210410_10wk_FBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1092M_20210406_9wk_RBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1093M_20210413_10wk_RBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1082M_20210405_10wk_FBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1060M_20210401_9wk_RBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1101M_20210402_9wk_FBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1062M_20210408_10wk_RBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1091M_20210330_8wk_RBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1074M_20210405_10wk_FBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1101M_20210413_10wk_RBG',
       '20210729_AJ_Toh_RatBloodGlucose_T1076M_20210329_9wk_FBG',
   

In [155]:
file_grouping[file_grouping['File Name'].isin(fr_scaled.index)]

,File Name,analytical_run_order,fr_name,rat_label,bg_type,quant_file,week,lipidex_file_number,date,glucose_tolerance
6,20210729_AJ_Toh_RatBloodGlucose_T1076M_2021040...,7,20210729_AJ_Toh_RatBloodGlucose_T1076M_2021040...,1076.0,FBG,True,10.0,27,2021-04-05,diabetic
7,20210729_AJ_Toh_RatBloodGlucose_T1062M_2021032...,8,20210729_AJ_Toh_RatBloodGlucose_T1062M_2021032...,1062.0,FBG,True,8.0,9,2021-03-22,impaired
8,20210729_AJ_Toh_RatBloodGlucose_T1093M_2021040...,9,20210729_AJ_Toh_RatBloodGlucose_T1093M_2021040...,1093.0,FBG,True,9.0,49,2021-04-02,normal
9,20210729_AJ_Toh_RatBloodGlucose_T1060M_2021032...,10,20210729_AJ_Toh_RatBloodGlucose_T1060M_2021032...,1060.0,RBG,True,8.0,4,2021-03-25,impaired
10,20210729_AJ_Toh_RatBloodGlucose_T1091M_2021041...,11,20210729_AJ_Toh_RatBloodGlucose_T1091M_2021041...,1091.0,FBG,True,10.0,39,2021-04-10,normal
11,20210729_AJ_Toh_RatBloodGlucose_T1092M_2021040...,12,20210729_AJ_Toh_RatBloodGlucose_T1092M_2021040...,1092.0,RBG,True,9.0,44,2021-04-06,impaired
12,20210729_AJ_Toh_RatBloodGlucose_T1093M_2021041...,13,20210729_AJ_Toh_RatBloodGlucose_T1093M_2021041...,1093.0,RBG,True,10.0,52,2021-04-13,normal
13,20210729_AJ_Toh_RatBloodGlucose_T1082M_2021040...,14,20210729_AJ_Toh_RatBloodGlucose_T1082M_2021040...,1082.0,FBG,True,10.0,33,2021-04-05,diabetic
14,20210729_AJ_Toh_RatBloodGlucose_T1060M_2021040...,15,20210729_AJ_Toh_RatBloodGlucose_T1060M_2021040...,1060.0,RBG,True,9.0,6,2021-04-01,impaired
15,20210729_AJ_Toh_RatBloodGlucose_T1101M_2021040...,16,20210729_AJ_Toh_RatBloodGlucose_T1101M_2021040...,1101.0,FBG,True,9.0,55,2021-04-02,diabetic


In [113]:
fr_scaled.iloc[:, 0:1].min()

0    0.0
dtype: float64

In [ ]:
## split data

x_train, x_test, y_train, y_test = train_test_split(features=fr_scaled, 
                                                    target=, 
                                                    test_size=0.2,
                                                    random_state=128792387,
                                                    )

In [4]:
svm_clf = svm.SVC(kernel='linear')  # linear is one of several possible kernels, can also use radial 

svm_clf.fit(x_train, y_train)

y_pred = svm_clf.predict(x_test)

In [86]:
from sklearn import datasets

#Load dataset
cancer = datasets.load_breast_cancer()

In [93]:
cancer.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,